In [1]:
import sys
import os
import keras_tuner
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Layer, Dropout
from tensorflow.keras import layers
from tensorflow.keras.layers import PReLU


# Get the absolute path of the current script's directory
current_dir = os.path.dirname(os.path.abspath("transformer0.ipynb"))

# Get the absolute path of the parent directory (project_folder)
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

from FNN1_1 import baseline_deviation, baeline_out_deviation, baseline_long_deviation, baseline_relError, absSum
baseline_out_deviation = baeline_out_deviation

from GetXY import x_train, y_train, x_val, y_val, x_test, y_test, out_x_test, out_y_test, long_x_test, long_y_test, outsideExpr, absSum

C:\Users\A_118784\Desktop\matura_github\myenv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\A_118784\Desktop\matura_github\myenv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\A_118784\Desktop\matura_github\myenv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. P

Epoch 1/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 21.1501 - val_loss: 16.8987
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 16.9835 - val_loss: 15.6992
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 15.9756 - val_loss: 14.7786
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14.7276 - val_loss: 13.3582
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.9624 - val_loss: 11.3199
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.4903 - val_loss: 8.5984
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.2999 - val_loss: 5.4554
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.0358 - val_loss: 2.7429
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.9869 - val_loss: 1.4448
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2499 - val_loss: 1.0897
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0275 - val_loss: 0.9503
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

C:\Users\A_118784\Desktop\matura_github\matura\matura\FNN1_1.py:246: RuntimeWarning: divide by zero encountered in divide
  relativeError = np.where(np.array(y_test) != 0, deviation.flatten() / np.abs(np.array(y_test)), deviation.flatten())


3 - 4 - -2
2543
1.0

Expressions not in x:
4 + 2 + -5
True
1457
1.0
15
-4.0
[-5.   1.   1.   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5]


In [2]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.0001,
    restore_best_weights=True,
    monitor='val_loss',
    mode = "min"
)

In [3]:
#defining the positional encoder modelled after the formula in the paper that was cited. (generated by gemini)
def posEncoding(max_seq_len, d_model):
    # Create a matrix of angles according to the formula
    angle_rads = get_angles(np.arange(max_seq_len)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
    
    # Apply sine to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    
    # Apply cosine to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
    # Add a batch dimension
    pos_encoding = angle_rads[np.newaxis, ...]
    
    return tf.cast(pos_encoding, dtype=tf.float32)

def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

In [4]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)

In [5]:
#edited by gemini
input_shape = x_train[0].shape
from tensorflow.keras.layers import Input, Flatten, Dense, PReLU, Dropout, Add, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

# --- Updated build_model function ---
def build_model(hp, input_shape):
    K.clear_session()
    
    # Hyperparameters
    num_neurons = hp.Int("num_neurons", 1, 512)
    num_layers = hp.Int("num_layers", 1, 16)
    dropoutTF = hp.Boolean("dropoutT/F")

    # --- Model Architecture ---
    inputs = Input(shape=input_shape) # Expects shape like (15,)

    # The input shape is (batch, 15). We need to treat it as a sequence.
    # Reshape to (batch, 15, 1) to represent a sequence of 15 steps with 1 feature each.
    # This is the key step to make shapes compatible.
    sequence_length = input_shape[0]
    d_model = 1
    x = Reshape((sequence_length, d_model))(inputs)

    # Add positional encoding
    positional_encoding = posEncoding(sequence_length, d_model)
    x = Add()([x, positional_encoding])
    
    # Flatten the sequence back into a vector of shape (batch, 15) for the Dense layers.
    x = Flatten()(x)

    # Add the dense layers as before
    for i in range(num_layers):
        x = Dense(num_neurons)(x)
        x = PReLU()(x)
        if dropoutTF:
            x = Dropout(0.1)(x)

    # Output Layer
    outputs = Dense(1, activation='linear')(x)

    # Create and compile the model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss="mse")

    return model
build_model(keras_tuner.HyperParameters(), input_shape)

<Functional name=functional, built=True>

In [6]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=lambda hp: build_model(hp, input_shape),    objective="val_loss",
    max_trials=50,
    executions_per_trial=1,
    overwrite=False,
    directory="FNN3Tuner",
    project_name="tuner3",
)

Reloading Tuner from FNN3Tuner\tuner3\tuner0.json


In [7]:
num_epochs = 100
tuner.search(train_dataset, epochs = num_epochs, validation_data = (val_dataset), verbose = 1, callbacks = [])

In [8]:
best_hps = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hps, input_shape)
best_model.summary()
tuner.results_summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 15)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 15, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ add (Add)                            │ (1, 15, 1)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (1, 15)                     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (1, 494)                    │           7,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ p_re_lu (PReLU)                      │ (1, 494)                    │             494 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (1, 1)                      │             495 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,893 (34.74 KB)

 Trainable params: 8,893 (34.74 KB)

 Non-trainable params: 0 (0.00 B)

Results summary
Results in FNN3Tuner\tuner3
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 14 summary
Hyperparameters:
num_neurons: 494
num_layers: 1
dropoutT/F: False
Score: 0.0013289122143760324

Trial 21 summary
Hyperparameters:
num_neurons: 349
num_layers: 1
dropoutT/F: False
Score: 0.0013864522334188223

Trial 35 summary
Hyperparameters:
num_neurons: 344
num_layers: 1
dropoutT/F: False
Score: 0.0014143552398309112

Trial 39 summary
Hyperparameters:
num_neurons: 155
num_layers: 1
dropoutT/F: False
Score: 0.0027343942783772945

Trial 42 summary
Hyperparameters:
num_neurons: 147
num_layers: 1
dropoutT/F: False
Score: 0.002782547613605857

Trial 19 summary
Hyperparameters:
num_neurons: 512
num_layers: 11
dropoutT/F: False
Score: 0.002827545627951622

Trial 25 summary
Hyperparameters:
num_neurons: 138
num_layers: 1
dropoutT/F: False
Score: 0.002850163960829377

Trial 36 summary
Hyperparameters:
num_neurons: 435
num_layers: 12
dropoutT/F: False
Score: 0.003326

In [9]:
x_test_dataset = tf.data.Dataset.from_tensor_slices(x_test).batch(batch_size)
out_x_test_dataset = tf.data.Dataset.from_tensor_slices(out_x_test).batch(batch_size)
long_x_test_dataset = tf.data.Dataset.from_tensor_slices(long_x_test).batch(batch_size)

In [10]:
best_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 17.9640 - val_loss: 16.8988
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14.6666 - val_loss: 14.9251
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.4946 - val_loss: 12.3810
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.4363 - val_loss: 8.7529
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.0935 - val_loss: 5.0205
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.2021 - val_loss: 2.5837
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.5767 - val_loss: 1.4030
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8814 - val_loss: 0.9523
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5936 - val_loss: 0.6463
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4144 - val_loss: 0.4579
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3035 - val_loss: 0.3284
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - los

In [11]:
predsInRange = best_model.predict(x_test_dataset)
predsOutRange = best_model.predict(out_x_test_dataset)
predsLongRange = best_model.predict(long_x_test_dataset)

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [12]:
import numpy as np

reldiffInRange = []
diffInRange = []
safe_y_test = np.where(np.isclose(y_test,0.0), 1.0, y_test)

for i in range(len(y_test)):
    diffInRange.append(abs(y_test[i] - predsInRange[i]))
    reldiffInRange.append(abs(y_test[i] - predsInRange[i])/abs(safe_y_test[i]))
print(len(diffInRange))
print("MAE in Range: ", np.mean(diffInRange))
print("MRE in Range: ", np.mean(reldiffInRange))

diffLongRange = []
for i in range(200, 300):
    diffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsLongRange[i])))
    
NEEDdiffLongRange = []
for i in range(len(long_y_test)):
    NEEDdiffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsLongRange[i])))
print("MAE longer Expressions: ", np.mean(NEEDdiffLongRange))

diffOutRange = []
for i in range(len(out_y_test)):
    diffOutRange.append(abs(out_y_test[i] - predsOutRange[i]))
safe_out_y_test = np.where(out_y_test == 0, 1, out_y_test)
diff_out_relError = []
for i in range(len(out_y_test)):
    diff_out_relError.append(abs(diffOutRange[i] / safe_out_y_test[i]))
print("MAE out Range: ", np.mean(diffOutRange))
print("MRE out Range: ", np.mean(diff_out_relError))

1457
MAE in Range:  0.02598668
MRE in Range:  0.010053354381036895
MAE longer Expressions:  5.600490063274545
MAE out Range:  2.3446362
MRE out Range:  0.2713937937780829


In [13]:
placeholder = absSum(outsideExpr)
diffOutRange = []
indices_with_placeholder_22 = [i for i, val in enumerate(placeholder) if val == 22] 

for i in indices_with_placeholder_22:
    diffOutRange.append(np.abs(out_y_test[i]-predsOutRange[i]))


In [14]:
meanDiff_InRange = np.mean(diffInRange)
meanDiff_OutRange = np.mean(diffOutRange)
meanDiff_LongRange = np.mean(diffLongRange)
meanDiff_OutRelRange = np.mean(diff_out_relError)



In [15]:
benchmark = 0
benchmark += baseline_deviation / (meanDiff_InRange**2) / 4
print(baseline_deviation / (meanDiff_InRange**2) / 4)

benchmark += baseline_out_deviation / (meanDiff_OutRange**2) / 4
print(baseline_out_deviation / (meanDiff_OutRange**2) / 4)

benchmark += baseline_long_deviation / (meanDiff_LongRange**2) / 4
print(baseline_long_deviation / (meanDiff_LongRange**2) / 4)

benchmark += baseline_relError / (meanDiff_OutRelRange**2) / 4
print(baseline_relError / (meanDiff_OutRelRange**2) / 4)

print(f"Benchmark: {benchmark}")

9.347200559139326
0.27120678357423184
0.08727067310572725
0.13472024556909531
Benchmark: 9.840398261388382


In [16]:
# New cell for RNN2 notebook - Statistical analysis with multiple runs
#this is also generated based on FNN6.py (DeepseekR1 this time.) (ofc, slightly tweaked as always)
# Initialize lists to store metrics across multiple runs
benchmarks_rnn = []
MAEinRange_rnn = []
MREinRange_rnn = []
MAEoutRange_rnn = []
MREoutRange_rnn = []
MAElongRange_rnn = []

# Run multiple training iterations
for progress in range(5):
    early_stopping = keras.callbacks.EarlyStopping(
        patience=10,
        min_delta=0.0001,
        restore_best_weights=True,
        monitor='val_loss',
        mode = "min"
    )
    print(f"Progress: {progress + 1}/5")
    
    # Build and train model
    current_model = build_model(best_hps, input_shape)
    
    current_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=200,
        callbacks=[early_stopping],
        verbose=1
    )
    
    # Make predictions
    predsInRange = current_model.predict(x_test_dataset, verbose=0)
    predsOutRange = current_model.predict(out_x_test_dataset, verbose=0)
    predsLongRange = current_model.predict(long_x_test_dataset, verbose=0)
    
    # Calculate metrics
    reldiffInRange = []
    diffInRange = []
    safe_y_test = np.where(np.isclose(y_test, 0.0), 1.0, y_test)
    
    for i in range(len(y_test)):
        diffInRange.append(abs(y_test[i] - predsInRange[i]))
        reldiffInRange.append(abs(y_test[i] - predsInRange[i])/abs(safe_y_test[i]))
    
    MAEinRange_rnn.append(np.mean(diffInRange))
    MREinRange_rnn.append(np.mean(reldiffInRange))

    # Long range metrics
    diffLongRange = []
    for i in range(200, 300):
        diffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsLongRange[i])))
    
    NEEDdiffLongRange = []
    for i in range(len(long_y_test)):
        NEEDdiffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsLongRange[i])))
    MAElongRange_rnn.append(np.mean(NEEDdiffLongRange))
    
    # Out of range metrics
    diffOutRange = []
    for i in range(len(out_y_test)):
        diffOutRange.append(abs(out_y_test[i] - predsOutRange[i]))
    safe_out_y_test = np.where(out_y_test == 0, 1, out_y_test)
    diff_out_relError = []
    for i in range(len(out_y_test)):
        diff_out_relError.append(abs(diffOutRange[i] / safe_out_y_test[i]))
    
    MAEoutRange_rnn.append(np.mean(diffOutRange))
    MREoutRange_rnn.append(np.mean(diff_out_relError))
    
    # Calculate benchmark
    placeholder = absSum(outsideExpr)
    diffOutRange_22 = []
    indices_with_placeholder_22 = [i for i, val in enumerate(placeholder) if val == 22] 
    
    for i in indices_with_placeholder_22:
        diffOutRange_22.append(np.abs(out_y_test[i]-predsOutRange[i]))
    
    meanDiff_InRange = np.mean(diffInRange)
    meanDiff_OutRange = np.mean(diffOutRange_22)
    meanDiff_LongRange = np.mean(diffLongRange)
    meanDiff_OutRelRange = np.mean(diff_out_relError)
    
    benchmark = 0
    benchmark += baseline_deviation / (meanDiff_InRange**2) / 4
    benchmark += baseline_out_deviation / (meanDiff_OutRange**2) / 4
    benchmark += baseline_long_deviation / (meanDiff_LongRange**2) / 4
    benchmark += baseline_relError / (meanDiff_OutRelRange**2) / 4
    
    benchmarks_rnn.append(benchmark)

# Statistical analysis
from scipy.stats import ttest_1samp


stats6, p_value6 = ttest_1samp(benchmarks_rnn, popmean=1)

print(f"FNN Model - Statistical Analysis Results:")

print(f"Benchmark P-value: {p_value6:.6f}")

print(f"\nFNN Model - Average Metrics:")
print(f"Average MAE in Range: {np.mean(MAEinRange_rnn):.6f}")
print(f"Average MRE in Range: {np.mean(MREinRange_rnn):.6f}")
print(f"Average MAE out Range: {np.mean(MAEoutRange_rnn):.6f}")
print(f"Average MRE out Range: {np.mean(MREoutRange_rnn):.6f}")
print(f"Average MAE long Range: {np.mean(MAElongRange_rnn):.6f}")
print(f"Average benchmark: {np.mean(benchmarks_rnn):.6f}")

print(f"\nFNN Model - All Runs:")
print(f"MAE in Range: {[f'{x:.6f}' for x in MAEinRange_rnn]}")
print(f"MRE in Range: {[f'{x:.6f}' for x in MREinRange_rnn]}")
print(f"MAE out Range: {[f'{x:.6f}' for x in MAEoutRange_rnn]}")
print(f"MRE out Range: {[f'{x:.6f}' for x in MREoutRange_rnn]}")
print(f"MAE long Range: {[f'{x:.6f}' for x in MAElongRange_rnn]}")
print(f"Benchmark: {[f'{x:.6f}' for x in benchmarks_rnn]}")

Progress: 1/5
Epoch 1/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 17.7717 - val_loss: 16.9476
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 15.0116 - val_loss: 15.5696
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 13.0346 - val_loss: 13.3368
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.1047 - val_loss: 9.1625
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.6478 - val_loss: 5.6834
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.6469 - val_loss: 2.9743
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.7169 - val_loss: 1.5116
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9511 - val_loss: 0.9871
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6042 - val_loss: 0.6439
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4094 - val_loss: 0.4635
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2877 - val_loss: 0.3358
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s